## 1. Import Dependencies

Load required libraries and utility functions.

In [1]:
# Restart kernel if needed to clear cached imports
from datetime import datetime
import json
from pathlib import Path
from sqlalchemy import create_engine, func

# Import utilities and models
import sys
sys.path.insert(0, '.')

from utils import reset_db, get_session, model_to_dict
from data.models.eventhub import (
    Base, User, Venue, Event, Reservation,
    Ticket, KBArticle, Conversation, Escalation
)

## 2. Database Configuration

Set up paths and database connection.

In [2]:
# Database path
DB_PATH = "data/db/eventhub.db"

# Data directory with JSONL files
DATA_DIR = Path("data/generated")

print(f"📁 Database: {DB_PATH}")
print(f"📂 Data source: {DATA_DIR}")

📁 Database: data/db/eventhub.db
📂 Data source: data\generated


## 3. Initialize Database

Reset the database and create fresh schema using SQLAlchemy models.

In [3]:
# Reset database (drops existing and recreates schema)
reset_db(DB_PATH, echo=False)

✅ Recreated data/db/eventhub.db with fresh schema


In [4]:
# Create SQLAlchemy engine
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)

# Create all tables from ORM models
Base.metadata.create_all(engine)

print("✅ Database schema created successfully")
print(f"📊 Tables: {list(Base.metadata.tables.keys())}")

✅ Database schema created successfully
📊 Tables: ['users', 'venues', 'events', 'reservations', 'tickets', 'kb_articles', 'conversations', 'escalations']


## 4. Load Data from JSONL Files

### 4.1 Load Venues

Load venue data including locations, capacity, and categories.

In [5]:
# Load venues from JSONL
venues_data = []
with open(DATA_DIR / "venues.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        venues_data.append(json.loads(line))

print(f"📥 Loaded {len(venues_data)} venues from JSONL")
print(f"\nSample venue:")
print(json.dumps(venues_data[0], indent=2))

📥 Loaded 40 venues from JSONL

Sample venue:
{
  "venue_id": "v_00001",
  "name": "The Independent",
  "address": "628 Divisadero Street",
  "neighborhood": "Divisadero",
  "city": "San Francisco",
  "state": "California",
  "capacity": 1602,
  "category": "music"
}


In [6]:
# Insert venues into database
with get_session(engine) as session:
    db_venues = []
    for venue_info in venues_data:
        venue = Venue(
            venue_id=venue_info["venue_id"],
            name=venue_info["name"],
            address=venue_info.get("address"),
            neighborhood=venue_info.get("neighborhood"),
            city=venue_info.get("city"),
            state=venue_info.get("state"),
            capacity=venue_info.get("capacity"),
            category=venue_info.get("category")
        )
        db_venues.append(venue)
    
    session.add_all(db_venues)

print(f"✅ Inserted {len(db_venues)} venues into database")

✅ Inserted 40 venues into database


### 4.2 Load Events

Load event data including dates, pricing, and availability.

In [7]:
# Load events from JSONL
events_data = []
with open(DATA_DIR / "events.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        events_data.append(json.loads(line))

print(f"📥 Loaded {len(events_data)} events from JSONL")
print(f"\nSample event:")
print(json.dumps(events_data[0], indent=2))

📥 Loaded 400 events from JSONL

Sample event:
{
  "event_id": "e_00001",
  "title": "The Independent Presents: Mac DeMarco - Live in SF",
  "description": "Indie sensation Mac DeMarco returns to The Independent with his unique brand of slacker rock. Don't miss this intimate performance!",
  "venue_id": "v_00001",
  "venue_name": "The Independent",
  "city": "San Francisco",
  "neighborhood": "Divisadero",
  "category": "music",
  "event_date": "2026-02-22",
  "start_time": "14:30",
  "duration_minutes": 120,
  "price_min": 34.0,
  "price_max": 92.0,
  "total_tickets": 1069,
  "tickets_sold": 207,
  "is_premium": false,
  "status": "cancelled"
}


In [8]:
# Insert events into database
with get_session(engine) as session:
    db_events = []
    for event_info in events_data:
        # Convert string date to date object if needed
        event_date = event_info.get("event_date")
        if isinstance(event_date, str):
            event_date = datetime.strptime(event_date, "%Y-%m-%d").date()
        
        event = Event(
            event_id=event_info["event_id"],
            title=event_info["title"],
            description=event_info.get("description"),
            venue_id=event_info.get("venue_id"),
            venue_name=event_info.get("venue_name"),
            city=event_info.get("city"),
            neighborhood=event_info.get("neighborhood"),
            category=event_info.get("category"),
            event_date=event_date,
            start_time=event_info.get("start_time"),
            duration_minutes=event_info.get("duration_minutes"),
            price_min=event_info.get("price_min"),
            price_max=event_info.get("price_max"),
            total_tickets=event_info.get("total_tickets"),
            tickets_sold=event_info.get("tickets_sold", 0),
            is_premium=event_info.get("is_premium", False),
            status=event_info.get("status", "active")
        )
        db_events.append(event)
    
    session.add_all(db_events)

print(f"✅ Inserted {len(db_events)} events into database")

✅ Inserted 400 events into database


### 4.3 Load Users

Load customer accounts with embedded subscription information.

In [9]:
# Load users from JSONL
users_data = []
with open(DATA_DIR / "users.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        users_data.append(json.loads(line))

print(f"📥 Loaded {len(users_data)} users from JSONL")
print(f"\nSample user:")
print(json.dumps(users_data[0], indent=2))

📥 Loaded 10000 users from JSONL

Sample user:
{
  "user_id": "u_00001",
  "full_name": "Liam \u00d3 Briain",
  "email": "liam.obriain@yahoo.ie",
  "city": "Dublin",
  "is_blocked": false,
  "created_at": "2024-12-02T21:40:03.563011",
  "subscription_tier": "premium",
  "subscription_status": "cancelled",
  "monthly_quota": 10,
  "subscription_started_at": "2024-12-13T21:40:03.563011",
  "subscription_ended_at": "2025-07-07T21:40:03.563011"
}


In [10]:
# Insert users into database (batch processing for performance)
BATCH_SIZE = 1000

with get_session(engine) as session:
    for i in range(0, len(users_data), BATCH_SIZE):
        batch = users_data[i:i+BATCH_SIZE]
        db_users = []
        
        for user_info in batch:
            # Convert string dates to datetime objects if needed
            created_at = user_info.get("created_at")
            if isinstance(created_at, str):
                created_at = datetime.fromisoformat(created_at.replace('Z', '+00:00'))
            
            subscription_started_at = user_info.get("subscription_started_at")
            if isinstance(subscription_started_at, str):
                subscription_started_at = datetime.fromisoformat(subscription_started_at.replace('Z', '+00:00'))
            
            subscription_ended_at = user_info.get("subscription_ended_at")
            if isinstance(subscription_ended_at, str):
                subscription_ended_at = datetime.fromisoformat(subscription_ended_at.replace('Z', '+00:00'))
            
            user = User(
                user_id=user_info["user_id"],
                full_name=user_info["full_name"],
                email=user_info["email"],
                city=user_info.get("city"),
                is_blocked=user_info.get("is_blocked", False),
                created_at=created_at,
                subscription_tier=user_info.get("subscription_tier", "basic"),
                subscription_status=user_info.get("subscription_status", "active"),
                monthly_quota=user_info.get("monthly_quota", 5),
                subscription_started_at=subscription_started_at,
                subscription_ended_at=subscription_ended_at
            )
            db_users.append(user)
        
        session.add_all(db_users)
        session.flush()  # Flush each batch
        print(f"   Processed batch {i//BATCH_SIZE + 1}: {len(db_users)} users")

print(f"\n✅ Inserted {len(users_data)} users into database")

   Processed batch 1: 1000 users
   Processed batch 2: 1000 users
   Processed batch 3: 1000 users
   Processed batch 4: 1000 users
   Processed batch 5: 1000 users
   Processed batch 6: 1000 users
   Processed batch 3: 1000 users
   Processed batch 4: 1000 users
   Processed batch 5: 1000 users
   Processed batch 6: 1000 users
   Processed batch 7: 1000 users
   Processed batch 8: 1000 users
   Processed batch 9: 1000 users
   Processed batch 7: 1000 users
   Processed batch 8: 1000 users
   Processed batch 9: 1000 users
   Processed batch 10: 1000 users

✅ Inserted 10000 users into database
   Processed batch 10: 1000 users

✅ Inserted 10000 users into database


### 4.4 Load Reservations

Load event bookings made by customers.

In [11]:
# Load reservations from JSONL
reservations_data = []
with open(DATA_DIR / "reservations.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        reservations_data.append(json.loads(line))

print(f"📥 Loaded {len(reservations_data)} reservations from JSONL")
print(f"\nSample reservation:")
print(json.dumps(reservations_data[0], indent=2))

📥 Loaded 5000 reservations from JSONL

Sample reservation:
{
  "reservation_id": "r_00001",
  "user_id": "u_05141",
  "user_email": "amuller@gmx.de",
  "event_id": "e_00388",
  "event_title": "Santa Clara University Baseball vs. Cal",
  "venue_id": "v_00039",
  "venue_name": "The Santa Clara University Track & Field Stadium",
  "event_date": "2025-12-18",
  "ticket_count": 2,
  "total_price": 196.38,
  "status": "confirmed",
  "booking_date": "2025-11-25",
  "payment_method": "apple_pay",
  "is_premium_booking": true
}


In [12]:
# Insert reservations into database (batch processing)
BATCH_SIZE = 1000

with get_session(engine) as session:
    for i in range(0, len(reservations_data), BATCH_SIZE):
        batch = reservations_data[i:i+BATCH_SIZE]
        db_reservations = []
        
        for res_info in batch:
            # Convert string dates
            event_date = res_info.get("event_date")
            if isinstance(event_date, str):
                event_date = datetime.strptime(event_date, "%Y-%m-%d").date()
            
            booking_date = res_info.get("booking_date")
            if isinstance(booking_date, str):
                booking_date = datetime.fromisoformat(booking_date.replace('Z', '+00:00'))
            
            reservation = Reservation(
                reservation_id=res_info["reservation_id"],
                user_id=res_info["user_id"],
                user_email=res_info.get("user_email"),
                event_id=res_info["event_id"],
                event_title=res_info.get("event_title"),
                venue_id=res_info.get("venue_id"),
                venue_name=res_info.get("venue_name"),
                event_date=event_date,
                ticket_count=res_info.get("ticket_count", 1),
                total_price=res_info.get("total_price"),
                status=res_info.get("status", "confirmed"),
                booking_date=booking_date,
                payment_method=res_info.get("payment_method"),
                is_premium_booking=res_info.get("is_premium_booking", False)
            )
            db_reservations.append(reservation)
        
        session.add_all(db_reservations)
        session.flush()
        print(f"   Processed batch {i//BATCH_SIZE + 1}: {len(db_reservations)} reservations")

print(f"\n✅ Inserted {len(reservations_data)} reservations into database")

   Processed batch 1: 1000 reservations
   Processed batch 2: 1000 reservations
   Processed batch 3: 1000 reservations
   Processed batch 4: 1000 reservations
   Processed batch 5: 1000 reservations

✅ Inserted 5000 reservations into database
   Processed batch 4: 1000 reservations
   Processed batch 5: 1000 reservations

✅ Inserted 5000 reservations into database


### 4.5 Load Support Tickets

Load customer support tickets across various categories.

In [13]:
# Load tickets from JSONL
tickets_data = []
with open(DATA_DIR / "tickets.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        tickets_data.append(json.loads(line))

print(f"📥 Loaded {len(tickets_data)} support tickets from JSONL")
print(f"\nSample ticket:")
print(json.dumps(tickets_data[0], indent=2))

📥 Loaded 505 support tickets from JSONL

Sample ticket:
{
  "ticket_id": "t_00001",
  "user_id": "u_01648",
  "user_email": "mia.mehta@gmail.com",
  "category": "general",
  "subject": "Accessibility information for Comedy Night",
  "description": "Hi, I'm attending The Comedy Night: The Tech Takeover on the 20th of this month with my friend. We both require wheelchair access. Could you please confirm if The Laughing Avocado is wheelchair accessible and if there are any special arrangements we need to make?",
  "status": "open",
  "priority": "medium",
  "created_at": "2025-11-29 18:00:43",
  "resolved_at": null,
  "agent_notes": null,
  "event_id": "e_00340",
  "event_title": "Comedy Night: The Tech Takeover"
}


In [14]:
# Insert tickets into database
with get_session(engine) as session:
    db_tickets = []
    for ticket_info in tickets_data:
        # Convert string dates
        created_at = ticket_info.get("created_at")
        if isinstance(created_at, str):
            created_at = datetime.fromisoformat(created_at.replace('Z', '+00:00'))
        
        resolved_at = ticket_info.get("resolved_at")
        if isinstance(resolved_at, str):
            resolved_at = datetime.fromisoformat(resolved_at.replace('Z', '+00:00'))
        
        ticket = Ticket(
            ticket_id=ticket_info["ticket_id"],
            user_id=ticket_info["user_id"],
            user_email=ticket_info.get("user_email"),
            category=ticket_info.get("category"),
            subject=ticket_info["subject"],
            description=ticket_info.get("description"),
            status=ticket_info.get("status", "open"),
            priority=ticket_info.get("priority", "medium"),
            created_at=created_at,
            resolved_at=resolved_at,
            agent_notes=ticket_info.get("agent_notes"),
            event_id=ticket_info.get("event_id"),
            event_title=ticket_info.get("event_title"),
            reservation_id=ticket_info.get("reservation_id")
        )
        db_tickets.append(ticket)
    
    session.add_all(db_tickets)

print(f"✅ Inserted {len(db_tickets)} support tickets into database")

✅ Inserted 505 support tickets into database


### 4.6 Load Knowledge Base Articles

Load self-service help articles for common questions.

In [15]:
# Load KB articles from JSONL
kb_articles_data = []
with open(DATA_DIR / "kb_articles.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        kb_articles_data.append(json.loads(line))

print(f"📥 Loaded {len(kb_articles_data)} KB articles from JSONL")
print(f"\nSample KB article:")
print(json.dumps(kb_articles_data[0], indent=2))

📥 Loaded 100 KB articles from JSONL

Sample KB article:
{
  "article_id": "kb_00001",
  "title": "How to Request a Refund",
  "content": "If you need to request a refund for your EventHub purchase, follow these steps:\n\n1. Navigate to your EventHub account on the website or app.\n2. Locate the order you'd like to cancel in your order history.\n3. Click 'Request Refund' next to the order.\n4. Choose a reason for the refund request and submit the form.\n\nImportant notes:\n- Refunds are processed within 7 business days, and you will be notified via email when the refund is complete.\n- EventHub's refund policy applies to cancellations made within the 24-hour window before the event start time.\n- Ticket insurance can be added during checkout to extend refund eligibility.\n\nContact us:\nIf you have any questions or issues with your refund, please contact our customer support team at support@eventhub.com.",
  "category": "how-to",
  "tags": [
    "refund",
    "money back",
    "cancel",

In [16]:
# Insert KB articles into database
with get_session(engine) as session:
    db_articles = []
    for article_info in kb_articles_data:
        # Convert string date
        last_updated = article_info.get("last_updated")
        if isinstance(last_updated, str):
            last_updated = datetime.fromisoformat(last_updated.replace('Z', '+00:00'))
        
        # Handle tags (convert list to JSON string if needed)
        tags = article_info.get("tags")
        if isinstance(tags, list):
            tags = json.dumps(tags)
        
        article = KBArticle(
            article_id=article_info["article_id"],
            title=article_info["title"],
            content=article_info["content"],
            category=article_info.get("category"),
            tags=tags,
            last_updated=last_updated,
            is_published=article_info.get("is_published", True),
            view_count=article_info.get("view_count", 0),
            helpful_votes=article_info.get("helpful_votes", 0)
        )
        db_articles.append(article)
    
    session.add_all(db_articles)

print(f"✅ Inserted {len(db_articles)} KB articles into database")

✅ Inserted 100 KB articles into database


## 5. Database Verification

Verify that all data was loaded correctly and examine statistics.

### 5.1 Record Counts

In [17]:
# Verify record counts for all tables
with get_session(engine) as session:
    print("📊 Database Record Counts:\n")
    print(f"  Users:         {session.query(User).count():,}")
    print(f"  Venues:        {session.query(Venue).count():,}")
    print(f"  Events:        {session.query(Event).count():,}")
    print(f"  Reservations:  {session.query(Reservation).count():,}")
    print(f"  Tickets:       {session.query(Ticket).count():,}")
    print(f"  KB Articles:   {session.query(KBArticle).count():,}")
    print(f"  Conversations: {session.query(Conversation).count():,} (empty - populated at runtime)")
    print(f"  Escalations:   {session.query(Escalation).count():,} (empty - populated at runtime)")

📊 Database Record Counts:

  Users:         10,000
  Venues:        40
  Events:        400
  Reservations:  5,000
  Tickets:       505
  KB Articles:   100
  Conversations: 0 (empty - populated at runtime)
  Escalations:   0 (empty - populated at runtime)


### 5.2 User Statistics

Analyze user demographics and subscription distribution.

In [18]:
# User subscription tier distribution
with get_session(engine) as session:
    print("👥 User Subscription Tiers:\n")
    
    tier_stats = session.query(
        User.subscription_tier,
        func.count(User.user_id).label('count')
    ).group_by(User.subscription_tier).all()
    
    for tier, count in tier_stats:
        percentage = (count / len(users_data)) * 100
        print(f"  {tier:10s}: {count:,} ({percentage:.1f}%)")
    
    # Subscription status
    print("\n📈 Subscription Status:\n")
    status_stats = session.query(
        User.subscription_status,
        func.count(User.user_id).label('count')
    ).group_by(User.subscription_status).all()
    
    for status, count in status_stats:
        percentage = (count / len(users_data)) * 100
        print(f"  {status:10s}: {count:,} ({percentage:.1f}%)")
    
    # Blocked users
    blocked_count = session.query(User).filter(User.is_blocked == True).count()
    print(f"\n🚫 Blocked users: {blocked_count:,}")

👥 User Subscription Tiers:

  basic     : 6,936 (69.4%)
  premium   : 3,064 (30.6%)

📈 Subscription Status:

  active    : 8,591 (85.9%)
  cancelled : 926 (9.3%)
  paused    : 483 (4.8%)

🚫 Blocked users: 122


### 5.3 Event Statistics

Examine event distribution by category and status.

In [19]:
# Event statistics
with get_session(engine) as session:
    print("🎭 Event Categories:\n")
    
    category_stats = session.query(
        Event.category,
        func.count(Event.event_id).label('count')
    ).group_by(Event.category).order_by(func.count(Event.event_id).desc()).all()
    
    for category, count in category_stats:
        print(f"  {category:20s}: {count:,} events")
    
    # Event status
    print("\n📊 Event Status:\n")
    status_stats = session.query(
        Event.status,
        func.count(Event.event_id).label('count')
    ).group_by(Event.status).all()
    
    for status, count in status_stats:
        print(f"  {status:10s}: {count:,} events")
    
    # Premium events
    premium_count = session.query(Event).filter(Event.is_premium == True).count()
    print(f"\n⭐ Premium events: {premium_count:,}")

🎭 Event Categories:

  music               : 120 events
  theater             : 100 events
  art                 : 60 events
  comedy              : 50 events
  sports              : 40 events
  conference          : 20 events
  museum              : 10 events

📊 Event Status:

  active    : 294 events
  cancelled : 38 events
  soldout   : 68 events

⭐ Premium events: 125


### 5.4 Reservation Statistics

Analyze booking patterns and revenue.

In [20]:
# Reservation statistics
with get_session(engine) as session:
    print("🎟️ Reservation Status:\n")
    
    status_stats = session.query(
        Reservation.status,
        func.count(Reservation.reservation_id).label('count')
    ).group_by(Reservation.status).all()
    
    for status, count in status_stats:
        print(f"  {status:15s}: {count:,} reservations")
    
    # Total revenue from confirmed bookings
    total_revenue = session.query(
        func.sum(Reservation.total_price)
    ).filter(Reservation.status == 'confirmed').scalar() or 0
    
    print(f"\n💰 Total confirmed revenue: ${total_revenue:,.2f}")
    
    # Average ticket count per reservation
    avg_tickets = session.query(
        func.avg(Reservation.ticket_count)
    ).scalar() or 0
    
    print(f"🎫 Average tickets per booking: {avg_tickets:.2f}")

🎟️ Reservation Status:

  cancelled      : 778 reservations
  confirmed      : 3,974 reservations
  pending        : 248 reservations

💰 Total confirmed revenue: $688,840.78
🎫 Average tickets per booking: 2.40


### 5.5 Support Ticket Statistics

Examine support ticket distribution by category, status, and priority.

In [21]:
# Support ticket statistics
with get_session(engine) as session:
    print("🎫 Support Ticket Status:\n")
    
    status_stats = session.query(
        Ticket.status,
        func.count(Ticket.ticket_id).label('count')
    ).group_by(Ticket.status).all()
    
    for status, count in status_stats:
        percentage = (count / len(tickets_data)) * 100
        print(f"  {status:15s}: {count:,} ({percentage:.1f}%)")
    
    print("\n📋 Ticket Categories:\n")
    category_stats = session.query(
        Ticket.category,
        func.count(Ticket.ticket_id).label('count')
    ).group_by(Ticket.category).order_by(func.count(Ticket.ticket_id).desc()).all()
    
    for category, count in category_stats:
        print(f"  {category:25s}: {count:,} tickets")
    
    print("\n⚡ Ticket Priorities:\n")
    priority_stats = session.query(
        Ticket.priority,
        func.count(Ticket.ticket_id).label('count')
    ).group_by(Ticket.priority).all()
    
    priority_order = {'urgent': 1, 'high': 2, 'medium': 3, 'low': 4}
    priority_stats = sorted(priority_stats, key=lambda x: priority_order.get(x[0], 99))
    
    for priority, count in priority_stats:
        percentage = (count / len(tickets_data)) * 100
        print(f"  {priority:10s}: {count:,} ({percentage:.1f}%)")

🎫 Support Ticket Status:

  escalated      : 45 (8.9%)
  in_progress    : 131 (25.9%)
  open           : 149 (29.5%)
  resolved       : 180 (35.6%)

📋 Ticket Categories:

  refund                   : 276 tickets
  technical                : 73 tickets
  cancellation             : 64 tickets
  general                  : 55 tickets
  complaint                : 37 tickets

⚡ Ticket Priorities:

  urgent    : 60 (11.9%)
  high      : 87 (17.2%)
  medium    : 251 (49.7%)
  low       : 107 (21.2%)


### 5.6 Top Events by Reservations

Identify the most popular events.

In [22]:
# Top 10 events by reservation count
with get_session(engine) as session:
    print("🏆 Top 10 Events by Reservations:\n")
    
    top_events = session.query(
        Event.title,
        Event.category,
        func.count(Reservation.reservation_id).label('bookings')
    ).join(
        Reservation, Event.event_id == Reservation.event_id
    ).group_by(
        Event.event_id
    ).order_by(
        func.count(Reservation.reservation_id).desc()
    ).limit(10).all()
    
    for idx, (title, category, bookings) in enumerate(top_events, 1):
        title_short = title[:40] + "..." if len(title) > 40 else title
        print(f"  {idx:2d}. {title_short:43s} [{category:15s}] - {bookings} bookings")

🏆 Top 10 Events by Reservations:

   1. The Exploratorium's Silent Disco Party      [museum         ] - 32 bookings
   2. Classical Music Night: The San Jose Symp... [music          ] - 29 bookings
   3. Monet's Water Lilies: A Bay Area Splash     [art            ] - 28 bookings
   4. Hip Hop History: A Tribute to Tupac Shak... [music          ] - 27 bookings
   5. Interactive Sculpture Garden: The Future... [art            ] - 27 bookings
   6. SF Shock Football Watch Party: Playoff G... [sports         ] - 26 bookings
   7. An Evening with Maya Angelou: A Literary... [theater        ] - 26 bookings
   8. An Evening with Silicon Valley Comedians... [theater        ] - 25 bookings
   9. Hip Hop Night: MC Thunder & The Beatboxe... [music          ] - 25 bookings
  10. Silicon Valley Nights: Techno & Beats wi... [music          ] - 25 bookings


### 5.7 Sample Data Queries

Test some common queries to verify relationships.

In [23]:
# Query a sample user with their reservations
with get_session(engine) as session:
    # Get a user with reservations
    user = session.query(User).join(
        Reservation, User.user_id == Reservation.user_id
    ).first()
    
    if user:
        print(f"👤 Sample User: {user.full_name} ({user.email})")
        print(f"   Subscription: {user.subscription_tier} ({user.subscription_status})")
        print(f"   Monthly quota: {user.monthly_quota}")
        
        # Get their reservations
        reservations = session.query(Reservation).filter(
            Reservation.user_id == user.user_id
        ).limit(3).all()
        
        print(f"\n   📅 Recent Reservations:")
        for res in reservations:
            print(f"      • {res.event_title[:40]:40s} - {res.status}")

👤 Sample User: Alexander Müller (amuller@gmx.de)
   Subscription: premium (active)
   Monthly quota: 10

   📅 Recent Reservations:
      • Santa Clara University Baseball vs. Cal  - confirmed


In [24]:
# Query a sample ticket with user and event details
with get_session(engine) as session:
    # Get an open ticket with event association
    ticket = session.query(Ticket).filter(
        Ticket.status == 'open',
        Ticket.event_id.isnot(None)
    ).first()
    
    if ticket:
        print(f"🎫 Sample Support Ticket: {ticket.ticket_id}")
        print(f"   User: {ticket.user_email}")
        print(f"   Category: {ticket.category}")
        print(f"   Priority: {ticket.priority}")
        print(f"   Subject: {ticket.subject}")
        print(f"   Status: {ticket.status}")
        if ticket.event_id:
            print(f"   Related Event: {ticket.event_title}")

🎫 Sample Support Ticket: t_00001
   User: mia.mehta@gmail.com
   Category: general
   Priority: medium
   Subject: Accessibility information for Comedy Night
   Status: open
   Related Event: Comedy Night: The Tech Takeover


## 6. Summary

Database initialization complete! The EventHub database is ready for the agentic support system.

### Next Steps:

1. **Set up RAG system** - Create vector embeddings for KB articles
2. **Build agent workflows** - Implement classifier, resolver, and escalation agents
3. **Create tools** - Database query tools, memory tools, RAG tools
4. **Test agents** - Run sample support scenarios

### Database Schema:
```
users (10,000 records)
  ├─ reservations (5,000 records) → events
  └─ tickets (500 records) → events

venues (40 records)
  └─ events (400 records)

kb_articles (100 records) - for RAG
conversations (runtime) - agent memory
escalations (runtime) - escalated cases
```

In [25]:
print("✅ EventHub Database Setup Complete!")
print(f"\n📁 Database location: {DB_PATH}")
print("🚀 Ready for agentic workflow integration")

✅ EventHub Database Setup Complete!

📁 Database location: data/db/eventhub.db
🚀 Ready for agentic workflow integration
